In [2]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker

In [4]:
ROLE='arn:aws:iam::720332985926:role/service-role/AmazonSageMaker-ExecutionRole-20240705T185967'
SOURCE_DIR='/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/src'

In [5]:
sagemaker_session = sagemaker.Session()

In [8]:
model_uri = 's3://sagemaker-us-east-1-720332985926/huggingface-pytorch-training-2025-06-19-23-24-05-737/output/model.tar.gz'

In [10]:
huggingface_model = HuggingFaceModel(
    entry_point='inference.py',
    model_data=model_uri,
    role=ROLE,
    transformers_version='4.49.0',
    py_version='py311',
    pytorch_version='2.5.1',
    sagemaker_session=sagemaker_session
    )